In [1]:
# 检查torch的安装以及gpu的使用
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# 检查MMAction2的安装
import mmaction
print(mmaction.__version__)

# 检查mmcv的安装
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu113 True
0.15.0
11.3
MSVC 192930137


In [2]:
from mmaction.apis import inference_recognizer, init_recognizer

# 选择tsn对应的配置文件
config = 'configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py'
# 加载上面下载的checkpoint文件
checkpoint =  'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# 初始化模型
print(config)
model = init_recognizer(config, checkpoint, device='cuda:0')

configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py


C:\Users\15281\anaconda3\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


load checkpoint from local path: checkpoints/swin_base_patch244_window877_kinetics400_1k.pth
The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 1024]) from checkpoint, the shape in current model is torch.Size([400, 2048]).


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random

# Function to read and randomly split a txt file
def read_and_split_txt(file_path, test_size=0.2):
    # Read the txt file into a pandas DataFrame
    data = pd.read_csv(file_path, delimiter=",", header=None)
    
    # Randomly shuffle the DataFrame
    data = data.sample(frac=1).reset_index(drop=True)
    
    # Split the data into features and target variable
    X = data  # Assuming the last column is the target variable
    y = data.iloc[:, -1]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    return X_train, X_test, y_train, y_test

# Example usage
file_path = './video_data/干净样本+噪声_A_分类.csv'
X_train, X_test, y_train, y_test = read_and_split_txt(file_path)
X_train.to_csv('./video_data/cu2/train.txt', index=False, header=False, sep=' ')
X_test.to_csv('./video_data/cu2/val_file.txt', index=False, header=False, sep=' ')
X_test.to_csv('./video_data/train.txt', index=False, header=False, sep=' ')
X_test.to_csv('./video_data/val_file1.txt', index=False, header=False, sep=' ')


In [4]:
from mmcv.runner import set_random_seed
# 获得tsn对应的配置文件cfg
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py')
# 修改数据集类型和各个文件路径
cfg.dataset_type = 'VideoDataset'
cfg.data_root = './video_data/train_video/'
cfg.data_root_val = './video_data/val_video/'
cfg.ann_file_train = './video_data/train.txt'
cfg.ann_file_val = './video_data/val_file1.txt'
cfg.ann_file_test = './video_data/val_file1.txt'


cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = './video_data/val_file1.txt'
cfg.data.test.data_prefix = './video_data/val_video/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = './video_data/train.txt'
cfg.data.train.data_prefix = './video_data/train_video/'
cfg.data.train.data_prefix = './video_data/train_video/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = './video_data/val_file1.txt'
cfg.data.val.data_prefix = './video_data/val_video/'

# 这里用于确认是否使用到omnisource训练
cfg.setdefault('omnisource', False)
# 修改cls_head中类别数为3
cfg.model.cls_head.num_classes = 3
# 使用预训练好的tsn模型
cfg.load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# 设置工作目录
cfg.work_dir = './tutorial_exps'

# 由于是单卡训练，修改对应的lr
#cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
#设置epoches
cfg.total_epochs = 20
# 设置存档点间隔减少存储空间的消耗
cfg.checkpoint_config.interval = 10
# 设置日志打印间隔减少打印时间
cfg.log_config.interval = 5

# 固定随机种子使得结果可复现
cfg.seed = 40
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(0,1)

# 打印所有的配置参数
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='Recognizer3D',
    backbone=dict(
        type='SwinTransformer3D',
        patch_size=(2, 4, 4),
        embed_dim=128,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        window_size=(8, 7, 7),
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        patch_norm=True),
    cls_head=dict(
        type='I3DHead',
        in_channels=1024,
        num_classes=3,
        spatial_type='avg',
        dropout_ratio=0.5),
    test_cfg=dict(average_clips='prob', max_testing_views=4))
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = './video_data/train_video/'
data_root_val = './v

In [5]:
#按排序重建训练文件
import pandas as pd
import numpy as np
import cv2
data = pd.read_csv("./video_data/干净样本+噪声.csv",sep=",", engine='python', encoding='utf-8')
data2 = pd.read_csv("./video_data/cu2/train.txt", delimiter=" ", header=None)
def select(data,data2,lr):
    ko = []
    for i in range(0,len(data2[0])):
        for j in range(0,len(data["名称"])):
            if(data2[0][i] == data["名称"][j]):
                if(data["dis"][j]<lr):
                    #print(i)
                    ko.append(i)
    data3 = data2.loc[ko, :]
    return data3
#data4 = select(data,data2,2.5)

In [6]:
import os.path as osp
import os
def clean():
    # Get a list of all files in the current directory
    files_in_directory = os.listdir()
    filtered_files = [file for file in files_in_directory if file.endswith(".pt")]
    # Loop through the list and remove each file
    for file in filtered_files:
        os.remove(file)
def cleancsv(directory):
    # 指定要清除CSV文件的目录
    # 遍历目录中的所有文件
    for filename in os.listdir(directory):
        # 检查文件扩展名是否为'.csv'
        if filename.endswith('.csv'):
            # 构造完整的文件路径
            file_path = os.path.join(directory, filename)
            # 删除文件
            os.remove(file_path)

In [7]:

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model,train_model_1

import mmcv
import mmcv
import os
import glob

# 构建动作识别模型
# 创建工作目录并训练模型
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
#datasets = [build_dataset(cfg.data.train)]
# 构建数据集
lr = 3
path = "./save/"
#train_model(model, datasets, cfg, distributed=False, validate= True)
for i in range(0,25):
    clean()
    cleancsv(path)
    lr = lr+1
    data4 = select(data,data2,lr)
    data4.to_csv('./video_data/train.txt', index=False, header=False, sep=' ')
    datasets = [build_dataset(cfg.data.train)]
    #model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
    cfg.lr_config = dict(policy='step', step=[40, 80])
    train_model(model, datasets, cfg, distributed=False, validate= True)
    #del model
    #torch.cuda.empty_cache()
    #del runner
    cfg.load_from = './tutorial_exps/epoch_20.pth'

C:\Users\15281\anaconda3\lib\site-packages\apex-0.1-py3.8.egg\apex\__init__.py:68: DeprecatedFeatureWarning: apex.amp is deprecated and will be removed by the end of February 2023. Use [PyTorch AMP](https://pytorch.org/docs/stable/amp.html)
2024-07-31 15:41:09,300 - mmaction - INFO - load checkpoint from local path: checkpoints/swin_base_patch244_window877_kinetics400_1k.pth
2024-07-31 15:41:09,431 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 1024]) from checkpoint, the shape in current model is torch.Size([3, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([3]).
2024-07-31 15:41:09,439 - mmaction - INFO - Start running, host: 15281@实验室, work_dir: H:\code\视频组代码\Video_Swin_Transformer\tutorial_exps
2024-07-31 15:41:09,439 - mmaction - INFO - Hooks will be executed in th

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


2024-07-31 15:41:27,632 - mmaction - INFO - Epoch [1][5/21]	lr: 7.813e-07, eta: 0:25:09, time: 3.637, data_time: 1.334, memory: 6119, top1_acc: 0.4000, top5_acc: 1.0000, loss_cls: 1.0989, loss: 1.0989
2024-07-31 15:41:34,484 - mmaction - INFO - Epoch [1][10/21]	lr: 7.813e-07, eta: 0:17:06, time: 1.370, data_time: 0.001, memory: 6804, top1_acc: 0.4000, top5_acc: 1.0000, loss_cls: 1.0239, loss: 1.0239
2024-07-31 15:41:41,249 - mmaction - INFO - Epoch [1][15/21]	lr: 7.813e-07, eta: 0:14:18, time: 1.353, data_time: 0.002, memory: 6813, top1_acc: 0.2000, top5_acc: 1.0000, loss_cls: 1.0951, loss: 1.0951
2024-07-31 15:41:47,987 - mmaction - INFO - Epoch [1][20/21]	lr: 7.813e-07, eta: 0:12:50, time: 1.348, data_time: 0.001, memory: 6814, top1_acc: 0.2000, top5_acc: 1.0000, loss_cls: 1.1191, loss: 1.1191
2024-07-31 15:42:04,150 - mmaction - INFO - Epoch [2][5/21]	lr: 7.813e-07, eta: 0:13:21, time: 2.871, data_time: 1.455, memory: 6817, top1_acc: 0.3000, top5_acc: 1.0000, loss_cls: 1.0737, loss:

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


2024-07-31 15:42:24,352 - mmaction - INFO - Epoch [2][20/21]	lr: 7.813e-07, eta: 0:11:15, time: 1.339, data_time: 0.002, memory: 6824, top1_acc: 0.4000, top5_acc: 1.0000, loss_cls: 1.0742, loss: 1.0742
2024-07-31 15:42:40,774 - mmaction - INFO - Epoch [3][5/21]	lr: 7.813e-07, eta: 0:11:36, time: 2.935, data_time: 1.529, memory: 6824, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.9220, loss: 0.9220
2024-07-31 15:42:47,547 - mmaction - INFO - Epoch [3][10/21]	lr: 7.813e-07, eta: 0:11:09, time: 1.355, data_time: 0.001, memory: 6824, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls: 1.0071, loss: 1.0071
2024-07-31 15:42:54,300 - mmaction - INFO - Epoch [3][15/21]	lr: 7.813e-07, eta: 0:10:45, time: 1.350, data_time: 0.002, memory: 6824, top1_acc: 0.3000, top5_acc: 1.0000, loss_cls: 1.1586, loss: 1.1586
2024-07-31 15:43:01,009 - mmaction - INFO - Epoch [3][20/21]	lr: 7.813e-07, eta: 0:10:23, time: 1.342, data_time: 0.001, memory: 6824, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 1.0191, loss

...

2024-08-01 07:57:36,969 - mmaction - INFO - Epoch [16][5/72]	lr: 7.813e-07, eta: 0:09:10, time: 3.089, data_time: 1.574, memory: 11758, top1_acc: 1.0000, top5_acc: 1.0000, loss_cls: 0.1198, loss: 0.1198
2024-08-01 07:57:44,452 - mmaction - INFO - Epoch [16][10/72]	lr: 7.813e-07, eta: 0:09:03, time: 1.496, data_time: 0.001, memory: 11758, top1_acc: 1.0000, top5_acc: 1.0000, loss_cls: 0.1020, loss: 0.1020
2024-08-01 07:57:51,793 - mmaction - INFO - Epoch [16][15/72]	lr: 7.813e-07, eta: 0:08:55, time: 1.468, data_time: 0.002, memory: 11758, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.2146, loss: 0.2146
2024-08-01 07:57:59,205 - mmaction - INFO - Epoch [16][20/72]	lr: 7.813e-07, eta: 0:08:47, time: 1.482, data_time: 0.002, memory: 11758, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls: 1.1406, loss: 1.1406
2024-08-01 07:58:06,722 - mmaction - INFO - Epoch [16][25/72]	lr: 7.813e-07, eta: 0:08:39, time: 1.504, data_time: 0.002, memory: 11758, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.

2024-08-01 08:03:06,301 - mmaction - INFO - Epoch [18][70/72]	lr: 7.813e-07, eta: 0:03:46, time: 1.429, data_time: 0.001, memory: 11758, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.1250, loss: 0.1250
2024-08-01 08:03:24,461 - mmaction - INFO - Epoch [19][5/72]	lr: 7.813e-07, eta: 0:03:35, time: 2.951, data_time: 1.390, memory: 11758, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.1851, loss: 0.1851
2024-08-01 08:03:31,777 - mmaction - INFO - Epoch [19][10/72]	lr: 7.813e-07, eta: 0:03:28, time: 1.463, data_time: 0.001, memory: 11758, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.3886, loss: 0.3886
2024-08-01 08:03:39,204 - mmaction - INFO - Epoch [19][15/72]	lr: 7.813e-07, eta: 0:03:20, time: 1.485, data_time: 0.001, memory: 11758, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.2752, loss: 0.2752
2024-08-01 08:03:46,736 - mmaction - INFO - Epoch [19][20/72]	lr: 7.813e-07, eta: 0:03:12, time: 1.506, data_time: 0.002, memory: 11758, top1_acc: 0.7000, top5_acc: 1.0000, loss_cls: 0.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 36/36, 0.4 task/s, elapsed: 103s, ETA:     0s

2024-08-01 08:08:43,072 - mmaction - INFO - Evaluating top_k_accuracy ...
2024-08-01 08:08:43,073 - mmaction - INFO - 
top1_acc	0.8056
top5_acc	1.0000
2024-08-01 08:08:43,073 - mmaction - INFO - Evaluating mean_class_accuracy ...
2024-08-01 08:08:43,074 - mmaction - INFO - 
mean_acc	0.8316
2024-08-01 08:08:43,075 - mmaction - INFO - Epoch(val) [20][36]	top1_acc: 0.8056, top5_acc: 1.0000, mean_class_accuracy: 0.8316


mean_class_accuracy
[0 1 1 2 2 0 2 1 0 1 2 0 2 2 1 0 1 1 2 2 1 1 1 2 0 2 1 1 2 2 2 2 1 0 0 2]
[0, 1, 1, 2, 2, 0, 1, 1, 0, 1, 0, 0, 2, 2, 1, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0, 2, 1, 2, 1, 2, 2, 1, 1, 0, 0, 1]


In [9]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
cfg.data.val.type = 'VideoDataset'
# 构建测试数据集
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 31/31, 0.4 task/s, elapsed: 86s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.9355
top5_acc	1.0000

Evaluating mean_class_accuracy ...
mean_class_accuracy
[1 2 1 0 2 0 2 1 1 1 1 1 2 2 1 0 2 0 1 2 2 1 2 2 1 0 2 2 1 1 1]
[1, 2, 1, 0, 2, 0, 2, 1, 1, 1, 1, 1, 2, 2, 1, 0, 2, 0, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 2]

mean_acc	0.9505
top1_acc: 0.9355
top5_acc: 1.0000
mean_class_accuracy: 0.9505


In [1]:
#打分矩阵
from sklearn.metrics import classification_report
import numpy as np
a = [2, 0, 1, 2, 1, 0, 1, 2, 0, 2, 2, 1, 0, 2, 1, 0, 1, 0, 1, 1, 1, 2, 2, 2, 0, 2, 1, 0, 1, 2, 2, 1, 1, 1]
b = [2, 0, 2, 2, 1, 0, 1, 2, 0, 2, 2, 1, 0, 2, 2, 0, 1, 0, 1, 1, 1, 2, 2, 2, 0, 2, 1, 0, 1, 2, 2, 1, 1, 1]
print(classification_report(a,b))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      0.86      0.92        14
           2       0.86      1.00      0.92        12

    accuracy                           0.94        34
   macro avg       0.95      0.95      0.95        34
weighted avg       0.95      0.94      0.94        34



In [8]:
torch.save(model, 'Swin-SPL-clean-noiseB.pth')

In [11]:
# 打开两个文件进行读取
with open('./video_data/cu2/train.txt', 'r') as file1, open('./video_data/cu/train2.txt', 'r') as file2:
    # 读取文件内容
    content1 = file1.read()
    content2 = file2.read()

# 将两个文件的内容拼接
combined_content = content1 + content2

# 将拼接后的内容写入到新的文件中
with open('./video_data/train_all.txt', 'w') as outfile:
    outfile.write(combined_content)

In [20]:
cfg.ann_file_val = './video_data/train_all.txt'
cfg.ann_file_test = './video_data/train_all.txt'
cfg.data.test.ann_file = 'train_all.txt'
cfg.data.test.data_prefix = './video_data/val_video/'
cfg.data.val.ann_file = './video_data/train_all.txt'
cfg.data.val.data_prefix = './video_data/val_video/'

In [21]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
cfg.data.val.type = 'VideoDataset'
# 构建测试数据集
path = "./save/"
cleancsv(path)
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
#eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 177/177, 0.6 task/s, elapsed: 315s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.9548
top5_acc	1.0000

Evaluating mean_class_accuracy ...
mean_class_accuracy
[2 2 1 1 1 2 0 2 1 2 1 1 1 1 1 2 2 1 0 1 2 1 1 2 2 2 0 1 0 0 1 2 0 2 2 2 0
 2 1 0 2 1 1 0 1 0 0 1 0 2 2 1 2 2 0 2 2 1 0 0 0 2 1 1 2 1 1 2 2 1 0 1 1 2
 2 0 0 0 1 0 1 2 2 1 1 2 2 1 1 2 0 2 2 2 0 2 1 2 2 1 0 2 0 0 1 0 1 2 0 0 1
 2 0 1 1 1 2 1 2 1 1 2 0 2 0 2 1 0 2 2 2 1 2 2 0 0 1 2 1 2 0 1 0 1 1 0 0 1
 0 0 1 2 1 2 0 2 1 0 2 0 2 0 1 1 0 0 1 0 1 0 0 2 2 2 0 2 2]
[2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 2, 2, 0, 1, 0, 0, 1, 2, 0, 2, 2, 2, 0, 2, 1, 0, 2, 1, 1, 0, 1, 0, 0, 1, 0, 2, 2, 1, 2, 2, 0, 2, 1, 1, 0, 0, 0, 2, 1, 1, 2, 1, 1, 2, 2, 1, 0, 1, 1, 2, 2, 0, 0, 0, 1, 0, 1, 2, 2, 1, 1, 2, 1, 1, 1, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 1, 0, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 2, 0, 2, 0, 2, 1, 0, 2, 2, 2, 1, 2, 2, 0, 0, 1, 2, 0, 2,

In [14]:
import os
import shutil

# 指定源目录和目标目录
source_dir = './save/'
target_dir = './save2/alltrain-c/'
cleancsv(target_dir)
# 遍历源目录
for filename in os.listdir(source_dir):
    # 检查文件扩展名是否为.csv
    if filename.endswith('.csv'):
        # 构建完整的文件路径
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)

        # 复制文件
        shutil.copy(src_path, dst_path)
        print(f'Copied {filename} to {target_dir}')

Copied 11_56_10.csv to ./save2/alltrain-c/
Copied 11_56_11.csv to ./save2/alltrain-c/
Copied 11_56_13.csv to ./save2/alltrain-c/
Copied 11_56_15.csv to ./save2/alltrain-c/
Copied 11_56_16.csv to ./save2/alltrain-c/
Copied 11_56_18.csv to ./save2/alltrain-c/
Copied 11_56_20.csv to ./save2/alltrain-c/
Copied 11_56_22.csv to ./save2/alltrain-c/
Copied 11_56_23.csv to ./save2/alltrain-c/
Copied 11_56_25.csv to ./save2/alltrain-c/
Copied 11_56_27.csv to ./save2/alltrain-c/
Copied 11_56_29.csv to ./save2/alltrain-c/
Copied 11_56_31.csv to ./save2/alltrain-c/
Copied 11_56_32.csv to ./save2/alltrain-c/
Copied 11_56_34.csv to ./save2/alltrain-c/
Copied 11_56_36.csv to ./save2/alltrain-c/
Copied 11_56_38.csv to ./save2/alltrain-c/
Copied 11_56_40.csv to ./save2/alltrain-c/
Copied 11_56_41.csv to ./save2/alltrain-c/
Copied 11_56_43.csv to ./save2/alltrain-c/
Copied 11_56_45.csv to ./save2/alltrain-c/
Copied 11_56_46.csv to ./save2/alltrain-c/
Copied 11_56_48.csv to ./save2/alltrain-c/
Copied 11_5

In [15]:
# 打开两个文件进行读取
with open('./video_data/cu2/val_file.txt', 'r') as file1, open('./video_data/cu/val_file2.txt', 'r') as file2:
    # 读取文件内容
    content1 = file1.read()
    content2 = file2.read()

# 将两个文件的内容拼接
combined_content = content1 + content2

# 将拼接后的内容写入到新的文件中
with open('./video_data/val_all.txt', 'w') as outfile:
    outfile.write(combined_content)

In [22]:
cfg.ann_file_val = './video_data/val_all.txt'
cfg.ann_file_test = './video_data/val_all.txt'
cfg.data.test.ann_file = 'train_all.txt'
cfg.data.test.data_prefix = './video_data/val_video/'
cfg.data.val.ann_file = './video_data/val_all.txt'
cfg.data.val.data_prefix = './video_data/val_video/'

In [23]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
cfg.data.val.type = 'VideoDataset'
# 构建测试数据集
path = "./save/"
cleancsv(path)
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

# 在测试集上评价训练完成的识别模型
eval_config = cfg.evaluation
#eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 45/45, 0.5 task/s, elapsed: 82s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.8444
top5_acc	1.0000

Evaluating mean_class_accuracy ...
mean_class_accuracy
[0 0 1 1 1 2 1 1 1 2 2 0 2 0 1 1 1 2 1 1 2 1 1 1 2 0 1 2 1 2 2 2 0 0 2 1 2
 0 2 2 1 1 1 2 2]
[0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 2, 1, 1, 0, 0, 2, 1, 2, 0, 2, 2, 1, 1, 0, 2, 2]

mean_acc	0.8550
top1_acc: 0.8444
top5_acc: 1.0000
mean_class_accuracy: 0.8550


In [19]:
import os
import shutil

# 指定源目录和目标目录
source_dir = './save/'
target_dir = './save2/allval-c/'
cleancsv(target_dir)
# 遍历源目录
for filename in os.listdir(source_dir):
    # 检查文件扩展名是否为.csv
    if filename.endswith('.csv'):
        # 构建完整的文件路径
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(target_dir, filename)

        # 复制文件
        shutil.copy(src_path, dst_path)
        print(f'Copied {filename} to {target_dir}')

Copied 12_1_25.csv to ./save2/allval-c/
Copied 12_1_27.csv to ./save2/allval-c/
Copied 12_1_28.csv to ./save2/allval-c/
Copied 12_1_30.csv to ./save2/allval-c/
Copied 12_1_32.csv to ./save2/allval-c/
Copied 12_1_34.csv to ./save2/allval-c/
Copied 12_1_35.csv to ./save2/allval-c/
Copied 12_1_37.csv to ./save2/allval-c/
Copied 12_1_39.csv to ./save2/allval-c/
Copied 12_1_41.csv to ./save2/allval-c/
Copied 12_1_43.csv to ./save2/allval-c/
Copied 12_1_44.csv to ./save2/allval-c/
Copied 12_1_46.csv to ./save2/allval-c/
Copied 12_1_48.csv to ./save2/allval-c/
Copied 12_1_50.csv to ./save2/allval-c/
Copied 12_1_51.csv to ./save2/allval-c/
Copied 12_1_53.csv to ./save2/allval-c/
Copied 12_1_55.csv to ./save2/allval-c/
Copied 12_1_57.csv to ./save2/allval-c/
Copied 12_1_58.csv to ./save2/allval-c/
Copied 12_2_0.csv to ./save2/allval-c/
Copied 12_2_11.csv to ./save2/allval-c/
Copied 12_2_13.csv to ./save2/allval-c/
Copied 12_2_14.csv to ./save2/allval-c/
Copied 12_2_16.csv to ./save2/allval-c/
C

In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pth')

In [ ]:
torch.save(model,'total_model.pth')